In [1]:
import numpy as np
import pandas as pd
import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 800)

import scipy

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import detrend

import datetime as dt

from sklearn.tree import DecisionTreeClassifier

Loading BokehJS ...

# Data

In [2]:
dat0 = pd.read_csv('data/ch4k_df.csv')
ref_dat0 = pd.read_csv('data/Article reference data.csv', low_memory = False, error_bad_lines = False, 
                       usecols = ['article_no', 'model_no', 'art_desc', 'sports_cat_desc', 'rmh_cat_desc', 
                                  'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc',
                                  'brand_desc', 'bus_unit_desc', 'rmh_cat_desc'])

# Remove clearance transactions!!
dat0['clearance'] = dat0.clearance.fillna(0) 
dat0['net_qty'] = (1 - dat0.clearance)*dat0.net_qty

# Magic

In [3]:
dat = dat0.copy()

# dat = dat[(dat.net_qty > 0) & (dat.season.isin(['FW15', 'FW16', 'FW17', 'FW18', 'FW19']))].round()

dat = dat[(dat.net_qty > 0) & (dat.season.isin(['SS17', 'SS18', 'SS19']))] # 'SS15', 'SS16', 

In [4]:
SS19 = pd.read_excel('data/ecom_SS19.xlsx').dropna()
SS19.shape

SS19 = SS19[SS19.carryover_FW18 == 'NO'] # new articles only


(7604, 3)

In [5]:
# Within article price, cost, margin averages
dat[['price', 'cost', 'margin']] = (
    dat.
    groupby('article_number', group_keys=False)[['price', 'cost', 'margin']].
    transform(lambda x: np.nanmean(x).round(2))
             )


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


In [6]:
# Remove articles where 'price = NA' in all transactions
dat = dat[~dat.price.isna()]


In [ ]:
dat.buy_availability.isna().sum()
(dat.buy_availability == 0).sum()

In [7]:
dat = dat.sort_values(['article_number', 'country', 'year', 'week'])

# ---- Replace NAs and zeros (w/ no impact replacements) ----
dat['buy_availability'] = dat.buy_availability.fillna(1) # assume full availability
dat['buy_availability'] = np.where(dat.buy_availability == 0, 1, dat.buy_availability) # replace 0 

In [8]:
# ---- Smooth buy_availability ----
def roll(df):
    return df.rolling(window = 5, min_periods = 1, center = True).mean()

# dat['buy_availability3'] = dat.groupby(['article_number', 'country'])['buy_availability3'].apply(roll)
dat['buy_availability'] = dat.groupby(['article_number', 'country'])['buy_availability'].apply(roll)

# ---- Corrected net_qty ----
dat['corr_net_qty'] = (dat.net_qty / dat.buy_availability).round()

In [9]:
# corr_season_net_qty = pd.DataFrame(dat.groupby(['article_number', 'season'], group_keys = False)['corr_net_qty'].apply(sum)).reset_index()
# dat['avg_buy_availability'] = dat.groupby(['article_number', 'season'])['buy_availability3'].transform(lambda x: np.nanmean(x).round(3))

dat['corr_season_net_qty'] = dat.groupby(['article_number', 'season'])['corr_net_qty'].transform(lambda x: np.sum(x))
dat['corr_season_net_qty'] = np.where(dat.corr_season_net_qty > dat.season_net_qty, dat.corr_season_net_qty, dat.season_net_qty) # only if >

dat['avg_buy_availability'] = dat.groupby(['article_number', 'season'])['buy_availability'].transform(lambda x: np.nanmean(x).round(3))



In [10]:

dat = (
    dat.
    sort_values(['article_number', 'season']).
    drop_duplicates(subset = 'article_number')
)

In [11]:
dat = dat[['article_number', 'brand', 'season', 'season_net_qty', 'corr_season_net_qty', 'avg_buy_availability', 
           'art_desc', 'sports_cat_desc', 'rmh_cat_desc', 'franchise', 
           'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc', 
           'price', 'cost', 'margin']]




In [12]:
dat = dat.set_index('article_number')


In [13]:
dat.shape
dat = dat[dat.corr_season_net_qty > 100]
dat.shape

(30615, 16)

(15842, 16)

In [14]:
# art = np.random.choice(dat.index, size = 500, replace = False)

# Just articles new in SS19, with season_net_qty > 200
art = set(SS19.article_number).intersection(set(dat.index))


In [ ]:
pd.crosstab(index = dat.season, columns = 'count')

In [15]:
from itertools import permutations 
from itertools import combinations

d = {}
cats = ['sports_cat_desc', 'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc']

# Create tidy dataframe, add results to it

for a in art:
    p = dat.loc[a, 'price']
    
    net_qtys = pd.Series()
    
    dat_p = dat[(dat.price >= 0.9*p) & (dat.price <= 1.1*p)]
    
    for c in cats: 
        dat_a = dat_p[dat_p[c] == dat_p.loc[a, c]] # filter to that level of that category        
        net_qtys = net_qtys.append(dat_a.corr_season_net_qty)
    
    for c2 in combinations(cats, 2):
        dat_a = dat_p[(dat_p[c2[0]] == dat_p.loc[a, c2[0]]) &
                      (dat_p[c2[1]] == dat_p.loc[a, c2[1]])]
        net_qtys = net_qtys.append(dat_a.corr_season_net_qty)
        
    for c3 in combinations(cats, 3):
        dat_a = dat_p[(dat_p[c3[0]] == dat_p.loc[a, c3[0]]) &
                      (dat_p[c3[1]] == dat_p.loc[a, c3[1]]) &
                      (dat_p[c3[2]] == dat_p.loc[a, c3[2]])]
        net_qtys = net_qtys.append(dat_a.corr_season_net_qty)
    
    for c4 in combinations(cats, 4):
        dat_a = dat_p[(dat_p[c4[0]] == dat_p.loc[a, c4[0]]) &
                      (dat_p[c4[1]] == dat_p.loc[a, c4[1]]) &
                      (dat_p[c4[2]] == dat_p.loc[a, c4[2]]) &
                      (dat_p[c4[3]] == dat_p.loc[a, c4[3]])]
        net_qtys = net_qtys.append(dat_a.corr_season_net_qty)
    
    for c4 in combinations(cats, 5):
        dat_a = dat_p[(dat_p[c4[0]] == dat_p.loc[a, c4[0]]) &
                      (dat_p[c4[1]] == dat_p.loc[a, c4[1]]) &
                      (dat_p[c4[2]] == dat_p.loc[a, c4[2]]) &
                      (dat_p[c4[3]] == dat_p.loc[a, c4[3]]) &
                      (dat_p[c4[4]] == dat_p.loc[a, c4[4]])]
        net_qtys = net_qtys.append(dat_a.corr_season_net_qty)
    
    for c4 in combinations(cats, 6):
        dat_a = dat_p[(dat_p[c4[0]] == dat_p.loc[a, c4[0]]) &
                      (dat_p[c4[1]] == dat_p.loc[a, c4[1]]) &
                      (dat_p[c4[2]] == dat_p.loc[a, c4[2]]) &
                      (dat_p[c4[3]] == dat_p.loc[a, c4[3]]) &
                      (dat_p[c4[4]] == dat_p.loc[a, c4[4]]) &
                      (dat_p[c4[5]] == dat_p.loc[a, c4[5]])]
        net_qtys = net_qtys.append(dat_a.corr_season_net_qty)
    
    d[a] = {
        'mean': net_qtys.mean(),
        'max': net_qtys.max(),
        '50': np.percentile(net_qtys, 50),
        '70': np.percentile(net_qtys, 70),
        '80': np.percentile(net_qtys, 80),
        '90': np.percentile(net_qtys, 90),
        'length': len(net_qtys)
           }
    if len(d) % 50 == 0:
        print(len(d))
        
        

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350


In [16]:
dat_art = dat[dat.index.isin(art)] # SS19-ers only


In [17]:
# Profit
def P(d, margin, cost, b):
    if d > b:    # CANNOT satisfy demand
        return b*margin
    
    elif d <= b: # CAN satisfy demand
        return d*margin - (b - d)*cost
    
    else:
        print('Error')

#### Save d

In [25]:
# save d
import json

json_i = json.dumps(d)
f = open("d.json", "w")
f.write(json_i)
f.close()

554979

In [ ]:
with open('d.json') as json_file:
    d_test = json.load(json_file)

# d.keys() == d_test.keys()

# {k: d[k] for k in sorted(d.keys())[:2]}

# {k: d_test[k] for k in sorted(d_test.keys())[:2]}

In [133]:
pct_l = '80'
pct_u = '90'

preds = pd.DataFrame([(a, d[a][pct_l], d[a][pct_u], d[a]['length']) for a in d.keys()]).round()
preds.columns = ('article_number', 'pctl_l', 'pctl_u', 'length')
preds = preds.set_index('article_number')

In [98]:
dat_art.head()
preds.head()

,brand,season,season_net_qty,corr_season_net_qty,avg_buy_availability,art_desc,sports_cat_desc,rmh_cat_desc,franchise,gender_desc,age_group_desc,prod_grp_desc,prod_type_desc,price,cost,margin
article_number,,,,,,,,,,,,,,,,
AC7438,adidas,SS19,614.0,954.0,0.696,BASELINE CMF INF,TENNIS,CORE SPORTS INSPIRED,GRAND COURT,UNISEX,INFANT,SHOES,SHOES - LOW (NON FOOTBALL),35.00,6.92,28.08
AC7548,adidas,SS19,124.0,124.0,0.998,UltraBOOST X S.,RUNNING,STELLA,ULTRA BOOST,WOMEN,ADULT,SHOES,SHOES - LOW (NON FOOTBALL),226.24,43.24,182.99
AC7980,adidas,SS19,229.0,233.0,0.985,TERREX SWIFT R2,OUTDOOR,OUTDOOR,NOT APPLICABLE,MEN,ADULT,SHOES,SHOES - LOW (NON FOOTBALL),120.10,24.03,96.07
AC8206,adidas,SS19,101.0,112.0,0.843,COPA 19.1 TF,FOOTBALL/SOCCER,FOOTBALL GENERIC,COPA,MEN,ADULT,SHOES,FOOTBALL SHOES (TURF),127.76,28.55,99.21
AH2131,adidas,SS19,282.0,321.0,0.957,SoleCourt M,TENNIS,TENNIS,NOT APPLICABLE,MEN,ADULT,SHOES,SHOES - LOW (NON FOOTBALL),159.35,41.46,117.89


,pctl_l,pctl_u,length
article_number,,,
DV2820,663.0,1990.0,9820
DX2461,358.0,1247.0,9467
DZ0053,303.0,1413.0,4980
DB3415,342.0,1665.0,13740
BB7216,273.0,1328.0,15197


In [134]:
dat_art0 = pd.merge(
    dat_art, preds,left_index = True, right_index = True
).merge(log_reg, left_index=True, right_index=True)

In [114]:
dat_art0.columns

Index(['brand', 'season', 'season_net_qty', 'corr_season_net_qty',
       'avg_buy_availability', 'art_desc', 'sports_cat_desc', 'rmh_cat_desc',
       'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc',
       'prod_type_desc', 'price', 'cost', 'margin', 'pctl_l', 'pctl_u',
       'length', '>500'],
      dtype='object')

In [135]:
# Logistic regression angle for <500 articles
dat_art0['pred'] = np.where(dat_art0['>500'], dat_art0['pctl_u'], dat_art0['pctl_l'])


In [50]:
dat_art2 = pd.merge(dat_art, preds, left_index=True, right_index=True,how = 'left')
dat_art2 = dat_art2[['season', 'season_net_qty', 'corr_season_net_qty', 'mean', 'pred_pct', 'length', 'art_desc', 'sports_cat_desc', 'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc', 'price', 'margin', 'cost']] # .sort_values('APE', ascending = False)
dat_art4 = pd.merge(dat_art2, SS19[['article_number', 'Ecom_FC_RMA']], left_index = True, right_on = 'article_number', how = 'left').drop('article_number', axis = 1)


In [51]:
#P(row['season_net_qty'], row['margin'], row['cost'], row['pred'])

dat_art4['eCom_profit'] = dat_art4.apply(lambda row: P(row['corr_season_net_qty'], row['margin'], row['cost'], row['Ecom_FC_RMA']), axis=1)
dat_art4['DAA_profit_pct']  = dat_art4.apply(lambda row: P(row['corr_season_net_qty'], row['margin'], row['cost'], row['pred']), axis=1)

# dat_art4['DAA_profit_pred']  = dat_art4.apply(lambda row: P(row['corr_season_net_qty'], row['margin'], row['cost'], row['pred']), axis=1)
# dat_art4['DAA_profit_m']  = dat_art4.apply(lambda row: P(row['corr_season_net_qty'], row['margin'], row['cost'], row['mean']), axis=1)



In [ ]:
# Profit w/ logistic regression component
dat_art00 = pd.merge(dat_art0, SS19[['article_number', 'Ecom_FC_RMA']], left_index = True, right_on = 'article_number', how = 'left').drop('article_number', axis = 1)

dat_art00['DAA_profit_pctl']  = dat_art00.apply(lambda row: P(row['corr_season_net_qty'], row['margin'], row['cost'], row['pctl_u']), axis=1)
dat_art00['eCom_profit'] = dat_art00.apply(lambda row: P(row['corr_season_net_qty'], row['margin'], row['cost'], row['Ecom_FC_RMA']), axis=1)
dat_art00['DAA_profit']  = dat_art00.apply(lambda row: P(row['corr_season_net_qty'], row['margin'], row['cost'], row['pred']), axis=1)

dat_art00[['DAA_profit', 'eCom_profit', 'DAA_profit_pctl']].sum().round()

# Using pctl_l = 50 for predicted <500 unit articles:
# --> eCom makes ~18m, we lose ~5m
# DAA_profit         108593975.0
# eCom_profit        131718527.0
# DAA_profit_pctl    106169547.0

# Using pctl_l = 70 for predicted <500 unit articles:
# --> eCom makes ~18m, we... MAKE MONEY!!
# DAA_profit         119232632.0
# eCom_profit        131718527.0
# DAA_profit_pctl    106169547.0

In [148]:
# Logistic regression approach profitability BY corrected net_qty bins
    
l = np.array([0, 500, 1000, 2000, 5000, 10000, 75000])
dat_art00.loc[:,'corr_seas_net_qty_bins'] = pd.cut(dat_art00['corr_season_net_qty'], bins = l)

dat_art00.groupby('corr_seas_net_qty_bins')[['DAA_profit', 'eCom_profit', 'DAA_profit_pctl']].sum().round()/1000000

,DAA_profit,eCom_profit,DAA_profit_pctl
corr_seas_net_qty_bins,,,
"(0, 500]",10.309045,19.144084,-7.490642
"(500, 1000]",25.977298,24.947239,23.091974
"(1000, 2000]",27.741250,27.226527,30.655019
"(2000, 5000]",28.509870,26.968363,31.618651
"(5000, 10000]",8.543577,10.534310,9.340675
"(10000, 75000]",18.704977,22.898003,18.953869


In [52]:
pct
dat_art4[(dat_art4.corr_season_net_qty > 500)][['DAA_profit_pct', 'eCom_profit']].sum().round()

# '90' # empirical percentile used
# DAA_profit_pct     113660189.0
# eCom_profit        112574442.0
# dtype: float64



'90'

DAA_profit_pct    113660189.0
eCom_profit       112574442.0
dtype: float64

In [ ]:
dat_art4['diff']  = dat_art4.apply(lambda row: row['pred_pct'] - row['corr_season_net_qty'], axis=1)


In [ ]:
dat_art4.columns

In [ ]:
l = np.array([0, 500, 1000, 2000, 5000, 10000, 75000])
dat_art4.loc[:,'prediction_bins'] = pd.cut(dat_art4['pred_pct'], bins = l)

# p = np.array([0, 50, 100, 150, 200, 300, 1000])
# dat_mini.loc[:,'price_bin'] = pd.cut(dat_mini.price, bins = p)

dat_art4.groupby('prediction_bins')[['DAA_profit_pct', 'eCom_profit']].apply(np.mean).round()

In [ ]:
dat_art4.groupby('prediction_bins')[['price']].apply(np.mean).round()

In [ ]:
d500['diff'].hist()

In [ ]:
x = 'corr_season_net_qty'
y = 'pred_pct'

plt.rcParams["figure.figsize"] = [12,6]
plt.scatter(dat_art4[x], dat_art4[y], alpha = 0.1)

plt.xlabel(x)
plt.ylabel(y)

x = np.linspace(0, 20000,100)
y = x
plt.plot(x, y, '-b')


In [ ]:
# Histogram

plt.hist(net_qtys, bins = [0, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 5000], density = True)

pass;

In [ ]:
# Empirical distributions

import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(net_qtys)

ecdf([100, 500, 1000, 3000])

np.percentile(net_qtys, 75)
net_qtys.describe()

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer 


## decision tree example

In [ ]:
cancer = load_breast_cancer() 

print("cancer.keys(): \n{}".format(cancer.keys()))
print()

print("Shape of cancer data: {}".format(cancer.data.shape))
print()

print("Sample counts per class:\n{}".format(
{n: v for n, v in zip(cancer.target_names, np.bincount(cancer.target))}))
print()

print("Feature names:\n{}".format(cancer.feature_names))



In [ ]:
type(cancer.data)
type(cancer.target)

In [ ]:


cancer = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify=cancer.target, random_state=42
)

In [ ]:
type(X_train)

In [ ]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)

print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

## decision tree article forecasting

### load and wrangle

In [ ]:
dat0 = pd.read_csv('data/ch4k_df.csv')
ref_dat0 = pd.read_csv('data/Article reference data.csv', low_memory = False, error_bad_lines = False, 
                       usecols = ['article_no', 'model_no', 'art_desc', 'sports_cat_desc', 'rmh_cat_desc', 
                                  'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc',
                                  'brand_desc', 'bus_unit_desc', 'rmh_cat_desc'])

# Remove clearance transactions!!
dat0['clearance'] = dat0.clearance.fillna(0) 
dat0['net_qty'] = (1 - dat0.clearance)*dat0.net_qty

In [ ]:
dat = dat0.copy()

# dat = dat[(dat.net_qty > 0) & (dat.season.isin(['FW15', 'FW16', 'FW17', 'FW18', 'FW19']))].round()

dat = dat[(dat.net_qty > 0) & (dat.season.isin(['SS17', 'SS18', 'SS19']))] # 'SS15', 'SS16', 

In [ ]:
SS19 = pd.read_excel('data/ecom_SS19.xlsx').dropna()
SS19.shape

SS19 = SS19[SS19.carryover_FW18 == 'NO'] # new articles only


In [ ]:
# Within article price, cost, margin averages
dat[['price', 'cost', 'margin']] = (
    dat.
    groupby('article_number', group_keys=False)[['price', 'cost', 'margin']].
    transform(lambda x: np.nanmean(x).round(2))
             )


In [ ]:
# Remove articles where 'price = NA' in all transactions
dat = dat[~dat.price.isna()]


In [ ]:
dat = dat.sort_values(['article_number', 'country', 'year', 'week'])

# ---- Replace NAs and zeros (w/ no impact replacements) ----
dat['buy_availability'] = dat.buy_availability.fillna(1) # assume full availability
dat['buy_availability'] = np.where(dat.buy_availability == 0, 1, dat.buy_availability) # replace 0 

In [ ]:
# ---- Smooth buy_availability ----
def roll(df):
    return df.rolling(window = 5, min_periods = 1, center = True).mean()

# dat['buy_availability3'] = dat.groupby(['article_number', 'country'])['buy_availability3'].apply(roll)
dat['buy_availability'] = dat.groupby(['article_number', 'country'])['buy_availability'].apply(roll)

# ---- Corrected net_qty ----
dat['corr_net_qty'] = (dat.net_qty / dat.buy_availability).round()

In [ ]:
# corr_season_net_qty = pd.DataFrame(dat.groupby(['article_number', 'season'], group_keys = False)['corr_net_qty'].apply(sum)).reset_index()
# dat['avg_buy_availability'] = dat.groupby(['article_number', 'season'])['buy_availability3'].transform(lambda x: np.nanmean(x).round(3))

dat['corr_season_net_qty'] = dat.groupby(['article_number', 'season'])['corr_net_qty'].transform(lambda x: np.sum(x))
dat['corr_season_net_qty'] = np.where(dat.corr_season_net_qty > dat.season_net_qty, dat.corr_season_net_qty, dat.season_net_qty) # only if >

dat['avg_buy_availability'] = dat.groupby(['article_number', 'season'])['buy_availability'].transform(lambda x: np.nanmean(x).round(3))



In [ ]:

dat = (
    dat.
    sort_values(['article_number', 'season']).
    drop_duplicates(subset = 'article_number')
)

In [ ]:
dat = dat[['article_number', 'brand', 'season', 'season_net_qty', 'corr_season_net_qty', 'avg_buy_availability', 
           'art_desc', 'sports_cat_desc', 'rmh_cat_desc', 'franchise', 
           'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc', 
           'price', 'cost', 'margin']]




In [ ]:
dat = dat.set_index('article_number')


In [ ]:
dat.shape
dat = dat[dat.corr_season_net_qty > 100]
dat.shape

In [ ]:
# art = np.random.choice(dat.index, size = 500, replace = False)

# Just articles new in SS19, with season_net_qty > 200
art = set(SS19.article_number).intersection(set(dat.index))


### fancy-ing

In [75]:
len(dat.index.unique())

15842

In [90]:
# mini = np.random.choice(dat.index, size = 5000, replace = False)
# dat_mini = dat[dat.index.isin(mini)].copy()

dat_tree = dat_art.copy()

In [91]:
# b = np.array([0, 500, 1000, 1500, 2000, 3000, 5000, 10000, 50000])
# dat_mini.loc[:,'net_qty_bin'] = pd.cut(dat_mini.season_net_qty, bins = b)

# p = np.array([0, 50, 100, 150, 200, 300, 1000])
# dat_mini.loc[:,'price_bin'] = pd.cut(dat_mini.price, bins = p)

dat_tree = dat[['corr_season_net_qty', 'sports_cat_desc',
       'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc',
       'prod_grp_desc', 'prod_type_desc', 'price']].copy()

In [92]:
dat_tree_dummies = pd.get_dummies(dat_tree)

X = dat_tree_dummies.loc[:, 'price':]
y = dat_tree_dummies.corr_season_net_qty > 500

In [87]:
### Logistic Regression

In [93]:
# Logistic regression on corr_net_qty > 500

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.values, y, random_state=42)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

logreg.score(X_train, y_train) 
logreg.score(X_test, y_test) # 0.74

# Could use this in combination with HS method to identify < 500 articles, then use lower percentile

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

0.738910866088713

0.7376925018934612

In [105]:
dat_tree_dummies.head()

,corr_season_net_qty,price,sports_cat_desc_AMERICAN FOOTBALL,sports_cat_desc_BASEBALL,sports_cat_desc_BASKETBALL,sports_cat_desc_BOXING,sports_cat_desc_CASUAL,sports_cat_desc_CYCLING,sports_cat_desc_DANCE,sports_cat_desc_FIELD HOCKEY,sports_cat_desc_FOOTBALL/SOCCER,sports_cat_desc_GOLF,sports_cat_desc_HANDBALL,sports_cat_desc_MIXED MARTIAL ARTS,sports_cat_desc_MOTORSPORT,sports_cat_desc_NOT SPORTS SPECIFIC,sports_cat_desc_OLYMPIC SPORTS,sports_cat_desc_ORIGINALS,sports_cat_desc_OUTDOOR,sports_cat_desc_RUGBY,sports_cat_desc_RUNNING,sports_cat_desc_SKATEBOARDING,sports_cat_desc_SNOWBOARDING,sports_cat_desc_STUDIO,sports_cat_desc_SWIM,sports_cat_desc_TENNIS,sports_cat_desc_TRACK AND FIELD,sports_cat_desc_TRAINING,sports_cat_desc_VOLLEYBALL,sports_cat_desc_WALKING,sports_cat_desc_WEIGHTLIFTING,sports_cat_desc_X-COUNTRY SKIING,sports_cat_desc_YOGA,rmh_cat_desc_ACTION SPORTS,rmh_cat_desc_AMERICAN FOOTBALL,rmh_cat_desc_ATHLETICS,rmh_cat_desc_BASEBALL/SOFTBALL,rmh_cat_desc_BASKETBALL GENERIC,rmh_cat_desc_BASKETBALL LICENSED,rmh_cat_desc_CLASSICS,rmh_cat_desc_CORE CLASSICS,rmh_cat_desc_CORE RUNNING,rmh_cat_desc_CORE SPORTS,rmh_cat_desc_CORE SPORTS INSPIRED,rmh_cat_desc_CORE TRAINING,rmh_cat_desc_DUMMY,rmh_cat_desc_FIELD SPORTS,rmh_cat_desc_FOOTBALL GENERIC,rmh_cat_desc_FOOTBALL LICENSED,rmh_cat_desc_GOLF,rmh_cat_desc_INDOOR SPORTS,rmh_cat_desc_ORIGINALS,rmh_cat_desc_OTHERS,rmh_cat_desc_OUTDOOR,rmh_cat_desc_PODIUM SPORTS,rmh_cat_desc_PORSCHE,rmh_cat_desc_RUNNING,rmh_cat_desc_SLIDE,rmh_cat_desc_STATEMENT,rmh_cat_desc_STELLA,rmh_cat_desc_SWIM,rmh_cat_desc_TENNIS,rmh_cat_desc_TRAINING,rmh_cat_desc_WALKING,rmh_cat_desc_YEEZY,franchise_25/7 TEE,franchise_4ATHLTS,franchise_4CMTE,franchise_ADIZERO,franchise_ADVANTAGE,franchise_ALL ME BRA,franchise_ALPHABOUNCE,franchise_ALPHASKIN TIGHT,franchise_ALTA,franchise_AZTREK,franchise_BARRICADE,franchise_BECKENBAUER TRACKTOP,franchise_BECKENBAUERTRACKPANT,franchise_BELIEVE THIS TIGHT,franchise_CAMPUS,franchise_CLASSIC BACKPACK,franchise_CLASSIC LEATHER,franchise_CLASSIC NYLON,franchise_CLUB C,franchise_CONTINENTAL 80,franchise_CONTROL HOODIE,franchise_COPA,franchise_CRAZY BYW,franchise_CRAZY EXPLOSIVE,franchise_CRAZY LIGHT,franchise_DEERUPT,franchise_DONT REST BRA,franchise_DURAMO,franchise_ENERGY BOOST,franchise_EPIC SHORT M,franchise_EPIC SHORT W,franchise_EQT ADV,franchise_FAST FLEXWEAVE,franchise_FIREBIRD TRACKPANTS,franchise_FIREBIRD TRACKTOP,franchise_FLOATRIDE RUN,franchise_FREELIFT TEE,franchise_FREESTYLE,franchise_GAZELLE,franchise_GRAND COURT,franchise_H90 CAP,franchise_HARDEN,franchise_HEARTRACER JACKET,franchise_HOW WE DO TIGHT,franchise_INSTAPUMP FURY,franchise_KRAFT SHORT,franchise_LILLARD,franchise_LUX TIGHT W,franchise_MARATHON 20 SHORT,franchise_MOVE TEE M,franchise_NANO,franchise_NEMEZIZ,franchise_NITE JOGGER,franchise_NIZZA,franchise_NMD,franchise_NOT APPLICABLE,franchise_OWN THE RUN SHORT,franchise_OWN THE RUN TEE,franchise_OWN THE RUN TIGHT,franchise_OWN THE RUN WIND JKT,franchise_OZWEEGO,franchise_P.O.D.SYSTEM,franchise_PARK,franchise_PHX JACKET,franchise_POWER BACKPACK,franchise_PREDATOR,franchise_PURE BOOST,franchise_PUREMOVE BRA,franchise_RAPIDA,franchise_RIVALRY,franchise_ROSE,franchise_RUNFALCON,franchise_SAMBA,franchise_SATURDAY SHORT,franchise_SATURDAY TIGHT,franchise_SOLAR BOOST,franchise_SOLE FURY,franchise_STABIL,franchise_STAN SMITH,franchise_STRIKER PANT,franchise_STRONGER FOR IT BRA,franchise_SUPERNOVA JACKET,franchise_SUPERSTAR,franchise_SUPERSTAR TRACKPANTS,franchise_SUPERSTAR TRACKTOP,franchise_TIRO PANT,franchise_TRAINING OPS BACKPCK,franchise_TRAINING TEAMBAG,franchise_TREFOIL HOODIE,franchise_TREFOIL TEE,franchise_ULTRA BOOST,franchise_VFA TEAMBAG,franchise_VL COURT,franchise_WND JACKET,franchise_WORKOUT,franchise_X,franchise_Z.N.E HOODIE,franchise_ZX FLUX,franchise_ZX TORSION,gender_desc_MEN,gender_desc_UNISEX,gender_desc_WOMEN,age_group_desc_ADULT,age_group_desc_INFANT,age_group_desc_JUNIOR,age_group_desc_KIDS,prod_grp_desc_APPAREL ACCESSORIES,prod_grp_desc_BA

In [112]:
log_reg = pd.DataFrame(data = logreg.predict(dat_tree_dummies.loc[:, 'price':]), index = dat_tree.index, columns = ['>500'])

# Optimal Buy

In [ ]:
# ---- Plot -----
# a = np.random.choice(preds.article_number.unique(), size = 1, replace = False)[0]
aoi = 'G26535'
a = aoi

dat_a = preds[preds.article_number == a][['week', 'net_qty', 'corrected', 'y_hat']]
        

plt.rcParams["figure.figsize"] = [10,7]
dat_a.sort_values('week').set_index('week').plot(linewidth = 3)
dat_a.sort_values('week').set_index('week').round()

dat_a[['net_qty', 'corrected', 'y_hat']].apply(np.sum).round()

preds_season.reset_index()[preds_season.index == a]

dat_aoi = dat0[dat0.article_number == aoi].copy()

dat_aoi = pd.merge(
    pd.DataFrame(dat_aoi.groupby(['year', 'week'])['net_qty'].sum()).reset_index(),
    dat_aoi[['year', 'week']].drop_duplicates()
)

dat_aoi.year = [str(x) for x in dat_aoi.year]
dat_aoi.week = [str(x) for x in dat_aoi.week]
dat_aoi['date'] = [dt.datetime.strptime(x[0] + '-' + x[1] + '-1', "%Y-%W-%w") for x in zip(dat_aoi.year, dat_aoi.week)]

plt.rcParams["figure.figsize"] = [10,7]
dat_aoi[['date', 'net_qty']].set_index('date').plot(linewidth = 4)

# dat_aoi

In [ ]:
dat[dat.index == art[0]][['season', 'season_net_qty', 'sports_cat_desc', 'rmh_cat_desc', 'franchise', 'price', 'margin', 'cost']]

minimize_EL(net_qtys, dat.loc[a, 'margin'], dat.loc[a, 'cost'])
net_qtys.describe()


In [ ]:
from functools import partial
from scipy import optimize
from scipy import integrate
import scipy.stats as stats

In [ ]:
# Loss --- demand, buy, margin, cost
def L(d, b, margin, cost):
    if d > b:    # CANNOT satisfy demand
        return (d - b)*margin
    
    elif d <= b: # CAN satisfy demand
        return (b - d)*cost
    
    else:
        print('Error')

# E[L | buy, article_mean, article_sd, article_margin, article_cost]
def EL(net_qtys, margin, cost, b):
    return sum([L(x, b, margin, cost) for x in net_qtys])/len(net_qtys)

def minimize_EL(net_qtys, margin, cost):
    p = partial(EL, net_qtys, margin, cost) # Make EL function of only one var
    mu = np.mean(net_qtys)
    buy_opt = optimize.minimize_scalar(p, bounds = (mu, mu + 2*np.std(net_qtys)))
    return int(buy_opt['x']) # optimal buy quantity

In [ ]:
# Profit
def P(d, margin, cost, b):
    if d > b:    # CANNOT satisfy demand
        return b*margin
    
    elif d <= b: # CAN satisfy demand
        return d*margin - (b - d)*cost
    
    else:
        print('Error')

def EP(net_qtys, margin, cost, b):
    return -sum([P(x, margin, cost, b) for x in net_qtys])Yeah /len(net_qtys) # integral--by rectangle method, each w/ 1/n height

def maximize_EP(net_qtys, margin, cost):
    p = partial(EP, net_qtys, margin, cost) # Make EL function of only one var
    mu = np.mean(net_qtys)
    buy_opt = optimize.minimize_scalar(p, bounds = (mu, mu + 2*np.std(net_qtys)))
    return int(buy_opt['x']) # optimal buy quantity

In [ ]:
# one offs

# # Minimize Loss
# l = partial(EL, net_qtys, 63, 30) # Make EL function of only one var
# [print(x, ':', round(l(x))) for x in range(0, 600, 50)]

# # Maximize Profit
# p = partial(EP, net_qtys, 63, 30)
# [print(x, ':', round(p(x))) for x in range(0, 600, 50)]


minimize_EL(net_qtys, 6, 2)
                          
maximize_EP(net_qtys, 6, 2)

# HUZZAH!!!!!


In [ ]:


# --- Overbuy statistics ---
# preds_season['pct_overbuy'] = (preds_season.Opt_Ovb - preds_season.y_hat)/preds_season.y_hat*100
# b = np.array([0, 1000, 2000, 5000, 10000, 50000])
# preds_season['bins'] = pd.cut(preds_season.y_hat, bins = b)
# preds_season.groupby('bins')['pct_overbuy'].describe().round()


# Appendix

In [ ]:
dat_art5[dat_art5.pred_90 < dat_art5.season_net_qty].sum()

# season_net_qty         227889.0
# pred_80                129573.0
# Ecom_FC_RMA            252412.0
# DAA_profit_pred_80    4725638.0
# eCom_profit           6433454.0
# diff                  2266284.0
# dtype: float64

# Comments: 
    # when we under-forecast... we DRAMATICALLY under-forecast
    # HUGE difference in profit

dat_art5[dat_art5.pred_90 >= dat_art5.season_net_qty].sum()

# season_net_qty         179877.0
# pred_80                354269.0
# Ecom_FC_RMA            310015.0
# DAA_profit_pred_80    4232213.0
# eCom_profit           4550809.0
# diff                  2017816.0
# dtype: float64

# Comments: 
    # in sum, we just slightly over-forecast
    # Tiny difference in profit
    

In [ ]:
## buy_availability correction

dat_roll = dat0.copy()

dat_roll = dat_roll[(dat_roll.net_qty > 0) & (dat_roll.season.isin(['SS17', 'SS18', 'SS19']))] # No buy_availability before '17


dat_roll = dat_roll[['article_number', 'country', 'season', 'year', 'week', 'net_qty', 'season_net_qty', 'art_desc', 'sports_cat_desc',
           'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc',
           'prod_grp_desc', 'prod_type_desc', 'price', 'margin', 'cost', 'buy_availability']]

dat_roll = dat_roll[dat_roll.season_net_qty > 200]

# dat_roll[['price', 'margin', 'cost']] = dat_roll[['price', 'margin', 'cost']] # .fillna(0).astype('int')



dat_roll = (
    dat_roll[['article_number', 'country', 'season', 'year', 'week', 'net_qty', 'buy_availability']].
    sort_values(['article_number', 'country', 'year', 'week'])
) 


dat_roll['buy_availability2'] = dat_roll.buy_availability.fillna(1) # assume full availability
dat_roll['buy_availability2'] = np.where(dat_roll.buy_availability2 == 0, 1, dat_roll.buy_availability2) # replace 0 

pd.crosstab(index = [dat_roll.season, dat_roll.buy_availability2], columns = 'counts')

rollers = np.random.choice(dat_roll.article_number, size = 5, replace = False)

dat_roll2 = dat_roll[dat_roll.article_number.isin(rollers)].copy()
dat_roll2 = dat_roll2.sort_values(['article_number', 'country', 'season', 'year', 'week'])

dat_roll2

# Smooth buy_availability

# Function
def roll(df):
    return df.rolling(window = 5, min_periods = 1, center = True).mean()

dat_roll2['buy_availability3'] = dat_roll2.groupby(['article_number', 'country'])['buy_availability2'].apply(roll)
# dat_roll2['buy_availability3'] = np.where(dat_roll2.buy_availability3 > 0.15, dat_roll2.buy_availability3, 0.15)


# Corrected net_qty
dat_roll2['corr_net_qty'] = (dat_roll2.net_qty / dat_roll2.buy_availability3).round()


In [ ]:
# for pct in ['pred', 'pred_50', 'pred_60', 'pred_70', 'pred_75', 'pred_80', 'pred_90', 'pred_98']:
#     col = 'DAA_profit' + '_' + pct
#     dat_art4[col]  = dat_art4.apply(lambda row: P(row['season_net_qty'], row['margin'], row['cost'], row[pct]), axis=1)
    
                          